### Todo

- INIT MODEL WITH CLASSIFIER MODEL WEIGHTS ✅
- FINE TUNE ✅
- ADD MORE AUGMENTATION (ZOOM ✅, CROP ON DIFF BACK ❌) ✅❌ 
- PREDICT WITH TTA 
- DISABLE BATCHNORM, BATCH-RENORM, USE ABN https://github.com/mapillary/inplace_abn

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import shutil
import os
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [2]:
sys.path.append("../fastai/old")
from fastai.conv_learner import *

In [3]:
torch.__version__

'0.3.1.post2'

### Train a detection/segmentation model for images with ships

**idea : rather than fine tuning imagenet for detection/segmentation fine tune with the fine tuned model for ship classifier** 

- train_seg_lbs : images in train_all folder with segmentation
- test_seg_lbs : images in test_all folder with segmentation

In [ ]:
path = Path("../DATA/airbus-ship/")
files = list(path.iterdir())

In [ ]:
files

### Recover segmentation data

In [ ]:
train_images = ! ls {path}/"train"
test_images = ! ls {path}/"test"

In [ ]:
train_segmentation = pd.read_csv(path/"train_ship_segmentations.csv")
test_segmentation = pd.read_csv(path/"test_ship_segmentations.csv")

In [ ]:
all_segmentation = pd.concat([train_segmentation, test_segmentation])

In [ ]:
trn_segmentation_df = all_segmentation[all_segmentation.ImageId.isin(train_images)]
test_segmentation_df = all_segmentation[all_segmentation.ImageId.isin(test_images)]

In [ ]:
trn_segmentation_df.shape, test_segmentation_df.shape

In [ ]:
# remove images without ship
trn_segmentation_df = trn_segmentation_df[~trn_segmentation_df.EncodedPixels.isna()]
test_segmentation_df = test_segmentation_df[~test_segmentation_df.EncodedPixels.isna()]

In [ ]:
os.makedirs(path/"segmentation", exist_ok=True)

In [ ]:
trn_segmentation_df.reset_index(drop=True).to_csv(path/"segmentation/trn_segmentation.csv", index=False)
test_segmentation_df.reset_index(drop=True).to_csv(path/"segmentation/test_segmentation.csv", index=False)

### Visualize segmentation data

We exclude images without ship to deal with imbalance and focus on learning ship segmentation...

In [ ]:
train_seg_lbs = pd.read_csv(path/"segmentation/trn_segmentation.csv")
test_seg_lbs = pd.read_csv(path/"segmentation/test_segmentation.csv")

In [ ]:
print(f"# of train segmentation images: {train_seg_lbs.ImageId.nunique()}")
print(f"# of test segmentation images: {test_seg_lbs.ImageId.nunique()}")

In [ ]:
print(f"# of train segmentation labels/ships: {len(train_seg_lbs.ImageId)}")
print(f"# of test segmentation labels/ships: {len(test_seg_lbs)}")

In [ ]:
from rle import rle_decode, rle_encode
from seg_plots import show_img_masks, plot_segmentation_df

In [ ]:
plot_segmentation_df(train_seg_lbs, path/'train', n=10, size=(10, 10))

### Save all masks 

Having rle_decode inside get_y is very expensive

In [ ]:
from tqdm import tqdm
from concurrent.futures import as_completed
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

def run_multi_progess(function, args_list, is_thread=True, **kwargs):
    # kwargs = {'unit': 'files','unit_scale': True, 'leave': True}
    if is_thread: executor = ThreadPoolExecutor(8)
    else: executor = ProcessPoolExecutor(4)
    with executor as e:
        futures = [e.submit(function, arg) for arg in args_list]
        for f in tqdm(as_completed(futures), total=len(futures), **kwargs):
            pass

In [ ]:
trn_unique_img_ids = train_seg_lbs.ImageId.unique()
test_unique_img_ids = test_seg_lbs.ImageId.unique()

In [ ]:
os.makedirs(path/'segmentation/train_masks', exist_ok=True)
os.makedirs(path/'segmentation/test_masks', exist_ok=True)

In [ ]:
trn_unique_img_ids[:4]

In [ ]:
def save_train_mask(img_id):
    rles = train_seg_lbs[train_seg_lbs.ImageId == img_id]['EncodedPixels'].values
    masks = sum([rle_decode(rle, (768, 768)) for rle in rles])
    np.save(str(path/'segmentation/train_masks'/img_id), masks)

In [ ]:
def save_test_mask(img_id):
    rles = test_seg_lbs[test_seg_lbs.ImageId == img_id]['EncodedPixels'].values
    masks = sum([rle_decode(rle, (768, 768)) for rle in rles])
    np.save(str(path/'segmentation/test_masks'/img_id), masks)

In [ ]:
run_multi_progess(save_train_mask, trn_unique_img_ids, unit=' images')

In [ ]:
run_multi_progess(save_test_mask, test_unique_img_ids, unit=' images')

### Dynamic unet segmentation Model

In [ ]:
from fastai.models.unet import *
from fastai.dataset import *
from fastai.core import *

In [ ]:
train_seg_lbs = pd.read_csv(path/"segmentation/trn_segmentation.csv")
test_seg_lbs = pd.read_csv(path/"segmentation/test_segmentation.csv")

In [ ]:
unique_img_ids = train_seg_lbs.ImageId.unique()
trn_fnames, val_fnames = train_test_split(unique_img_ids, test_size=0.1, random_state=42)

In [ ]:
test_fnames = test_seg_lbs.ImageId.unique()

In [ ]:
TRN_X = [f"train/{fname}" for fname in trn_fnames]
TRN_Y = [f"segmentation/train_masks/{fname}.npy" for fname in trn_fnames]

VAL_X = [f"train/{fname}" for fname in val_fnames]
VAL_Y = [f"segmentation/train_masks/{fname}.npy" for fname in val_fnames]

TEST_X = [f"test/{fname}" for fname in test_fnames]
TEST_Y = [f"segmentation/test_masks/{fname}.npy" for fname in test_fnames]

In [ ]:
test_sub_fnames = list((path/"test_v2").glob("*.jpg"))

In [ ]:
TEST_SUB_X = [f"test_v2/{fname.name}" for fname in test_sub_fnames]

In [ ]:
TEST_SUB_X[:3]

In [ ]:
len(TRN_X), len(VAL_X), len(TEST_X), len(TEST_SUB_X)

In [ ]:
TRN_Y[:4]

In [ ]:
TEST_Y[:4]

In [ ]:
np.load(path/TEST_Y[0])

### Check masks

- Some masks are not separated

In [ ]:
nrows = 3
fig, axes = plt.subplots(nrows, 10, figsize=(nrows*10, 10))
for i, ax in enumerate(axes.flatten()): 
    fname = np.random.permutation(VAL_Y)[i]
    ax.imshow(np.load(path/fname))
    ax.set_title(fname.split("/")[-1])
    
fig.tight_layout()

In [ ]:
class FilesEncodedDataset(BaseDataset):
    def __init__(self, fnames, fnames2, transform, path):
        self.fnames = fnames
        self.fnames2 = fnames2
        self.path = path
        super().__init__(transform)
    
    def get_sz(self): return self.transform.sz
    def get_x(self, i): return open_image(os.path.join(self.path, self.fnames[i]))
    def get_y(self, i): 
        
        mask = np.load(os.path.join(self.path, self.fnames2[i])).astype('float32')
        #mask = cv2.resize(mask, (sz, sz)).astype('float32')
        #mask = np.round(mask)
        return mask
        
    def get_n(self): return len(self.fnames)
    def get_c(self): return 0

    def resize_imgs(self, targ, new_path):
        dest = resize_imgs(self.fnames, targ, self.path, new_path)
        return self.__class__(self.fnames, self.y, self.transform, dest)

    def denorm(self,arr):
        """Reverse the normalization done to a batch of images.

        Arguments:
            arr: of shape/size (N,3,sz,sz)
        """
        if type(arr) is not np.ndarray: arr = to_np(arr)
        if len(arr.shape)==3: arr = arr[None]
        return self.transform.denorm(np.rollaxis(arr,1,4))

In [ ]:
class Transform():
    """ A class that represents a transform.

    All other transforms should subclass it.
    All subclasses should override
    do_transform.

    Arguments
    ---------
        tfm_y : TfmType
            type of transform
    """
    def __init__(self, tfm_y=TfmType.NO):
        self.tfm_y=tfm_y
        self.store = threading.local()

    def set_state(self): pass
    def __call__(self, x, y):
        self.set_state()
        x,y = ((self.transform(x),y) if self.tfm_y==TfmType.NO
                else self.transform(x,y) if self.tfm_y in (TfmType.PIXEL, TfmType.CLASS)
                else self.transform_coord(x,y))
        return x, y

    def transform_coord(self, x, y): return self.transform(x),y

    def transform(self, x, y=None):
        x = self.do_transform(x,False)
        return (x, self.do_transform(y,True)) if y is not None else x

    @abstractmethod
    def do_transform(self, x, is_y): raise NotImplementedError

In [ ]:
class RandomLighting(Transform):
    def __init__(self, b, c, tfm_y=TfmType.NO):
        super().__init__(tfm_y)
        self.b,self.c = b,c

    def set_state(self):
        self.store.b_rand = rand0(self.b)
        self.store.c_rand = rand0(self.c)

    def do_transform(self, x, is_y):
        if is_y and self.tfm_y != TfmType.PIXEL: return x
        b = self.store.b_rand
        c = self.store.c_rand
        c = -1/(c-1) if c<0 else c+1
        x = lighting(x, b, c)
        return x

In [ ]:
class RandomDihedral(CoordTransform):
    """
    Rotates images by random multiples of 90 degrees and/or reflection.
    Please reference D8(dihedral group of order eight), the group of all symmetries of the square.
    """
    def set_state(self):
        self.store.rot_times = random.randint(0,3)
        self.store.do_flip = random.random()<0.5

    def do_transform(self, x, is_y):
        x = np.rot90(x, self.store.rot_times)
        return np.fliplr(x).copy() if self.store.do_flip else x

In [ ]:
def rotate_cv(im, deg, mode=cv2.BORDER_CONSTANT, interpolation=cv2.INTER_AREA):
    """ Rotate an image by deg degrees

    Arguments:
        deg (float): degree to rotate.
    """
    r,c,*_ = im.shape
    M = cv2.getRotationMatrix2D((c//2,r//2),deg,1)
    return cv2.warpAffine(im,M,(c,r), borderMode=mode, flags=cv2.WARP_FILL_OUTLIERS+interpolation)

In [ ]:
class RandomRotate(CoordTransform):
    """ Rotates images and (optionally) target y.

    Rotating coordinates is treated differently for x and y on this
    transform.
     Arguments:
        deg (float): degree to rotate.
        p (float): probability of rotation
        mode: type of border
        tfm_y (TfmType): type of y transform
    """
    def __init__(self, deg, p=0.75, mode=cv2.BORDER_REFLECT, tfm_y=TfmType.NO):
        super().__init__(tfm_y)
        self.deg,self.p = deg,p
        if tfm_y == TfmType.COORD or tfm_y == TfmType.CLASS:
            self.modes = (mode,cv2.BORDER_CONSTANT)
        else:
            self.modes = (mode,mode)

    def set_state(self):
        self.store.rdeg = rand0(self.deg)
        self.store.rp = random.random()<self.p

    def do_transform(self, x, is_y):
        if self.store.rp: x = rotate_cv(x, self.store.rdeg, 
                mode= self.modes[1] if is_y else self.modes[0],
                interpolation=cv2.INTER_NEAREST)
        return x

In [ ]:
def zoom_cv(x,z):
    """ Zoom the center of image x by a factor of z+1 while retaining the original image size and proportion. """
    if z==0: return x
    r,c,*_ = x.shape
    M = cv2.getRotationMatrix2D((c/2,r/2),0,z+1.)
    return cv2.warpAffine(x,M,(c,r), borderMode=cv2.BORDER_CONSTANT, flags=cv2.WARP_FILL_OUTLIERS+cv2.INTER_NEAREST)

In [ ]:
class RandomZoom(CoordTransform):
    def __init__(self, zoom_max, zoom_min=0, mode=cv2.BORDER_REFLECT, tfm_y=TfmType.NO, p=1):
        super().__init__(tfm_y)
        self.zoom_max, self.zoom_min = zoom_max, zoom_min
        self.p = p

    def set_state(self):
        self.store.zoom = self.zoom_min+(self.zoom_max-self.zoom_min)*random.random()
        self.store.rp = random.random()<self.p
        
    def do_transform(self, x, is_y):
        if self.store.rp:
            x = zoom_cv(x, self.store.zoom)
        return x

In [ ]:
#RandomRotate(deg=30, p=0.7, tfm_y=TfmType.PIXEL)
f = vgg16
sz = 768
tfms = tfms_from_model(f,
                       sz,
                       aug_tfms=[
                                 RandomRotate(20, p=0.2, mode=cv2.BORDER_REFLECT, tfm_y=TfmType.PIXEL),
                           
                                 RandomDihedral(tfm_y=TfmType.PIXEL),
                           
                                 RandomZoom(zoom_max=1.5, zoom_min=0, mode=cv2.BORDER_CONSTANT,
                                            tfm_y=TfmType.PIXEL, p=0.2),
                                 
                                 RandomBlur(blur_strengths=3, probability=0.2, tfm_y=TfmType.NO),
                                 
                                 RandomLighting(0.05, 0.05)],
                       
                       tfm_y=TfmType.PIXEL,
                       norm_y=False,
                       crop_type=CropType.NO) 

In [ ]:
dataset = ImageData.get_ds(FilesEncodedDataset, 
                           trn=(TRN_X, TRN_Y),
                           val=(VAL_X, VAL_Y), tfms=tfms,
                           test=(TEST_X, TEST_Y) ,path=path)

In [ ]:
md = ImageData(path, dataset, bs=16, num_workers=8, classes=None)

In [ ]:
denorm = md.trn_ds.denorm

In [ ]:
*x, y= next(iter(md.trn_dl))

In [ ]:
# plot augmentations
i = 1

fig_sz = (20, 20)
fig, axes = plt.subplots(2, 4, figsize=fig_sz)
for ax in np.rollaxis(axes, -1):
    x,y = next(iter(md.aug_dl))
    img = denorm(x)[i]
    ax[0].imshow(img)
    ax[1].imshow(to_np(y[i]))

In [ ]:
np.unique(to_np(y))

In [ ]:
print(f"n train: {len(md.trn_ds)}, n val: {len(md.val_ds)}, n test: {len(md.test_ds)}")

### Model

In [ ]:
# load defined model# load  
def get_encoder(f, cut):
    base_model = (cut_model(f(True), cut))
    return nn.Sequential(*base_model)

def get_model(f=resnet18, sz=128):
    """gets dynamic unet model"""
    # cut encoder
    cut, cut_lr = model_meta[f]

    # define encoder
    encoder = get_encoder(f, cut)

    # init model
    # binary: ship - not ship
    m = DynamicUnet(encoder, n_classes=1) 

    # init upsample on cpu
    inp = torch.ones(1, 3, sz, sz)
    out = m(V(inp).cpu())

    # put model to gpu if desired# put mo 
    m = m.cuda(0)
    return m

In [ ]:
def dice_loss(logits, target):
    logits = torch.sigmoid(logits)
    smooth = 1.0

    iflat = logits.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    
    return ((2.0 * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth))

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, logits, target):
        logits = logits.squeeze(1)
        probas = torch.sigmoid(logits)
        pt = (target)*probas + (1 - target)*(1 - probas)
        loss = (-(1 - pt)**gamma)*torch.log(pt)
        return loss.mean()

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, logits, target):
        logits = logits.squeeze(1)
        if not (target.size() == logits.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), logits.size()))

        max_val = (-logits).clamp(min=0)
        loss = logits - logits * target + max_val + \
            ((-max_val).exp() + (-logits - max_val).exp()).log()

        invprobs = F.logsigmoid(-logits * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        
        return loss.mean()

In [ ]:
class BCELoss2D(nn.Module):
    def __init__(self):
        super(BCELoss2D, self).__init__()
        
    def forward(self, logits, targets):
        logits = logits.squeeze(1)
        logits = F.sigmoid(logits)
        return F.binary_cross_entropy(logits, targets)

In [ ]:
class MixedLoss(nn.Module):
    def __init__(self, alpha, gamma):
        super().__init__()
        self.alpha = alpha
        self.focal = FocalLoss(gamma)
        
    def forward(self, input, target):
        loss = self.alpha*self.focal(input, target) - torch.log(dice_loss(input, target))
        return loss.mean()

In [ ]:
class UpsampleModel():
    def __init__(self, model, cut_lr, name='upsample'):
        self.model,self.name, self.cut_lr = model, name, cut_lr

    def get_layer_groups(self, precompute):
        lgs = list(split_by_idxs(children(self.model.encoder), [self.cut_lr]))
        return lgs + [children(self.model)[1:]]

### Test metric

ref: https://www.kaggle.com/stkbailey/step-by-step-explanation-of-scoring-metric

In [ ]:
from skimage.measure import label
from eval_metric import sigmoid, get_gt_masks, create_iou_matrix, f2_IOU, get_pred_masks

In [ ]:
def single_image_score(labels, gt_rles, gt):
    """
    return avg thresholded f2 score for single image
    labels : labeled image array
    gt_rles : array of rles
    """
    if len(np.unique(labels)) == 1:
        if gt is None: 
            """original image has no instance"""
            return 1
        else:
            """no prediction is made tp = 0"""
            return 0
    else:
        pred_mask_arrays = get_pred_masks(labels)
        gt_mask_arrays = get_gt_masks(gt_rles)
        IOU = create_iou_matrix(pred_mask_arrays, gt_mask_arrays)
        return f2_IOU(IOU)

In [ ]:
shift = 0 # shift to keep track of file index 
n_valids = len(md.val_ds) # total # of validation samples
fnames = md.val_ds.fnames # validation filenames
df = train_seg_lbs

def fastai_metric(preds, targs):
    global shift
    global df
    global fnames
    global n_valids
    
    mask_thresh = 0.5 
    n_x = len(preds)

    scores = [] 
    preds = (sigmoid(to_np(preds).squeeze(1)) > mask_thresh).astype('uint8')
    gts = to_np(targs)
    
    for i, (gt_i, pred_i) in enumerate(zip(gts, preds)):
        fname = fnames[i+shift]
        gt_rles = df[df.ImageId == fname.split("/")[-1]]['EncodedPixels'].values
        labels = label(pred_i)
        scores.append(single_image_score(labels, gt_rles, gt_i))

    shift += n_x
    if shift == n_valids: shift = 0
    return np.mean(scores)

### Train with eval metric

In [ ]:
init_model = False
f = resnet18
cut, cut_lr = model_meta[f]
model = get_model(f, sz=768)
models = UpsampleModel(model, cut_lr)

if init_model:
    cls_weights = torch.load(path/"models/resnet34_classification_ft_v2.224.h5")
    state_dict_keys = list(model.encoder.state_dict().keys())
    for k in state_dict_keys: model.encoder.state_dict()[k].copy_(cls_weights[k])

In [ ]:
learn = ConvLearner(md, models)
learn.opt_fn=optim.Adam
learn.crit = MixedLoss(10, 2)
learn.metrics = [fastai_metric]

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()

In [47]:
learn.fit(3e-3, n_cycle=1, cycle_len=10, use_clr=(20, 10))

epoch      trn_loss   val_loss   fastai_metric                  
    0      0.398282   0.383816   0.246809  
    1      0.373837   0.312183   0.279532                       
    2      0.303733   4.703829   0.267776                       
    3      0.329927   0.36759    0.31621                        
    4      0.314626   0.256905   0.314863                       
    5      0.253966   0.247697   0.330798                       
    6      0.238425   0.238802   0.34247                        
    7      0.267847   0.221725   0.357736                       
    8      0.213951   0.218158   0.356572                       
    9      0.22161    0.202039   0.364887                       



[array([0.20204]), 0.36488680514216265]

In [50]:
learn.save("resnet34_segmentation_v6.768")

In [ ]:
learn.lr_find()

 49%|████▊     | 1044/2148 [18:04<33:12,  1.80s/it, loss=0.375] 

In [ ]:
learn.sched.plot()

In [ ]:
learn.fit(3e-3, n_cycle=1, cycle_len=10, use_clr=(20, 10))

In [ ]:
learn.save("resnet34_segmentation_v7.768")

### Fine tuning

In [ ]:
learn.load("resnet34_segmentation_v4.768")

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
learn.unfreeze()
lr = 1e-4
lrs = [lr/100, lr/10, lr]
learn.fit(lrs, n_cycle=1, cycle_len=8)

### Visualize predictions

In [ ]:
learn.load("resnet34_segmentation_v3.768")

In [ ]:
from seg_plots import plot_batch

In [ ]:
plot_batch(path,
           learn.model.eval(),
           learn.data.val_dl,
           learn.data.val_ds.fnames,
           20)

### Evaluate single test preds

In [ ]:
from seg_plots import show_img_masks

In [ ]:
test_preds = learn.predict(is_test=True)

In [ ]:
test_preds2 = [sigmoid(pred[0]) for pred in test_preds]

In [ ]:
len(test_preds2)

In [ ]:
test_seg_lbs.head(2)

In [ ]:
pred_fnames = iter(zip(test_preds2, learn.data.test_ds.fnames))

In [ ]:
pred, fname = next(pred_fnames)
fname = fname.split("/")[1]    
gt_rles = test_seg_lbs[test_seg_lbs.ImageId == fname]['EncodedPixels'].values
f2_score = single_image_score(label(pred>0.5), gt_rles, gt_rles)
print(f2_score)
show_img_masks(open_image(path/f"test/{fname}"), pred>0.5, size=(20, 20))

In [ ]:
score_maps = []
for pred, fname in zip(test_preds2, learn.data.test_ds.fnames):
    fname = fname.split("/")[1]    
    gt_rles = test_seg_lbs[test_seg_lbs.ImageId == fname]['EncodedPixels'].values
    f2_score = single_image_score(label(pred > 0.4), gt_rles, gt_rles)
    score_maps.append((len(gt_rles), f2_score))

In [ ]:
test_eval_df = pd.DataFrame(score_maps, columns=["n_ships", "f2score"])

In [ ]:
test_eval_df.groupby("n_ships")["f2score"].mean()

In [ ]:
test_eval_df["f2score"].mean()

In [ ]:
0.97*0.52 + 0.38*0.48

In [ ]:
img = (test_preds2[18]>0.5).astype(np.uint8)
plt.imshow(img)

In [ ]:
ret, thresh = cv2.threshold(img, 0.5, 1, 0)

In [ ]:
img, contours, hierarchy = cv2.findContours(img, 1, 2)

In [ ]:
cnt = contours[0]

In [ ]:
# center (x,y), (width, height), angle of rotation 
rect = cv2.minAreaRect(cnt)

In [ ]:
bbox_wh = cv2.boxPoints(rect)
bbox_wh = np.int0(bbox_wh)
angle = rect[-1]

In [ ]:
bbox_wh, angle

In [ ]:
x,y = np.mean(bbox_wh[::2], 0)
w = int(np.sqrt(np.mean((bbox_wh[1] - bbox_wh[0])**2)))
h = int(np.sqrt(np.mean((bbox_wh[2] - bbox_wh[1])**2)))

In [ ]:
def draw_rbbox(bbox, angle, ax, col='red'):
    """min row, min col, max row, max col - like np """
    min_y, min_x, max_y, max_x = bbox
    ax.add_patch(Rectangle((min_x, min_y),
                           max_x - min_x,
                           max_y - min_y,
                           angle=angle,
                           fill=False,
                           color=col))

In [ ]:
def draw_rbbox2(x,y,w,h,angle, ax, col='red'):
    """min row, min col, max row, max col - like np """
    ax.add_patch(Rectangle((x, y),
                           w,
                           h,
                           angle=angle,
                           fill=False,
                           color=col))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
ax.imshow(img)
ax.scatter(*np.int0(np.mean(bbox_wh[:2], 0)), c="red")
ax.scatter(*np.int0(np.mean(bbox_wh[2:], 0)), c="red")
ax.scatter(*np.int0(np.mean(bbox_wh[::2], 0)), c="red")
draw_rbbox2(x,y,h,w,angle, ax)

### Segmentation submission

In [ ]:
from rle import rle_encode

In [ ]:
classification_preds = pd.read_csv(path/"classification/test_preds_resnet34_classification_ft_v2_224.csv", usecols=[1,2])

In [ ]:
classification_preds.head()

In [ ]:
thresh = 0.4
sum(classification_preds.has_ship_proba > thresh)

In [ ]:
has_ship_images = classification_preds[classification_preds.has_ship_proba > thresh]["ImageId"].values

In [ ]:
test_seg_fnames = [f"test_v2/{fname}" for fname in has_ship_images]

In [ ]:
TEST_SUB_X = test_seg_fnames
TEST_SUB_Y = TRN_Y[:len(test_seg_fnames)]

In [ ]:
len(TEST_SUB_X)

In [ ]:
#RandomRotate(deg=30, p=0.7, tfm_y=TfmType.PIXEL)
f = resnet34
sz = 768
tfms = tfms_from_model(f,
                       sz,
                       aug_tfms=[RandomDihedral(tfm_y=TfmType.PIXEL),
                                 RandomBlur(),
                                 RandomLighting(0.05, 0.05)],
                       tfm_y=TfmType.PIXEL,
                       norm_y=False,
                       crop_type=CropType.NO) 

In [ ]:
dataset = ImageData.get_ds(FilesEncodedDataset, 
                           trn=(TRN_X, TRN_Y),
                           val=(VAL_X, VAL_Y), tfms=tfms,
                           test=(TEST_SUB_X, TEST_SUB_Y) ,path=path)

In [ ]:
md = ImageData(path, dataset, bs=4, num_workers=8, classes=None)

In [ ]:
f = resnet34
cut, cut_lr = model_meta[f]
model = get_model(f, sz=768)
models = UpsampleModel(model, cut_lr)

learn = ConvLearner(md, models)
learn.opt_fn=optim.Adam
learn.crit = MixedLoss(10, 2)
learn.metrics = [fastai_metric]

In [ ]:
learn.load("resnet34_segmentation_v3.768")

In [ ]:
test_preds = learn.predict(is_test=True)

In [ ]:
test_preds = [sigmoid(pred.squeeze(0)) for pred in test_preds]
test_fnames = [fname.split("/")[1] for fname in learn.data.test_ds.fnames]

In [ ]:
len(test_preds), len(test_fnames)

In [ ]:
plt.imshow(open_image(path/f"test_v2/{test_fnames[6]}"))

In [ ]:
plt.imshow(test_preds[6] > 0.5)

In [ ]:
long_preds = []
for fname, pred in zip(test_fnames, test_preds):
    labels = label(pred > 0.4)
    pred_masks = get_pred_masks(labels, sz=None)
    if pred_masks == []:
        long_preds.append((fname, None))
    else:
        for pred_mask in pred_masks:
            long_preds.append((fname, rle_encode(pred_mask)))

In [ ]:
segmentation_preds = pd.DataFrame(list(zip(*long_preds))).T.\
                rename(columns={0:"ImageId", 1:"EncodedPixels"})

In [ ]:
segmentation_preds.ImageId.nunique()

In [ ]:
no_ship_img_ids = classification_preds[classification_preds.has_ship_proba <=thresh]["ImageId"].values

In [ ]:
no_ship_df = pd.DataFrame({"ImageId": no_ship_img_ids, "EncodedPixels":None})

In [ ]:
submission_df = pd.concat([no_ship_df, segmentation_preds])

In [ ]:
path

In [ ]:
submission_df.to_csv(path/"submission/resnet_34_cls_ft_resnet_34_seg_init.csv", index=False)

In [ ]:
FileLink(path/"submission/resnet_34_cls_ft_resnet_34_seg_init.csv")